# Experiment Tracking with MLflow
Practical demonstration of tracking, logging parameters, metrics, and artifacts.

In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)
with mlflow.start_run():
    n_estimators = 100
    clf = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    acc = accuracy_score(y_test, preds)
    mlflow.log_param('n_estimators', n_estimators)
    mlflow.log_metric('accuracy', acc)
    mlflow.sklearn.log_model(clf, 'model')
    print(f'Accuracy: {acc:.2f}')

# Model Registry
Register the best model in the MLflow Model Registry and promote it between 'Staging' and 'Production' in the UI.

In [ ]:
model_name = 'Iris-RandomForest-Best'
with mlflow.start_run():
    n_estimators = 100
    clf = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    clf.fit(X_train, y_train)
    mlflow.sklearn.log_model(clf, 'model', registered_model_name=model_name)
    print(f'Model registered in the Model Registry with the name: {model_name}')